In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.stats.api as sms
from scipy import stats
import matplotlib.pyplot as plt
import plotly.express as px
import scipy
import seaborn as sns
import matplotlib.font_manager as fm
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from statsmodels.stats.outliers_influence import variance_inflation_factor
import math
import warnings
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectFromModel
import random


Bad key text.latex.preview in file C:\Users\spa84\anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle, line 123 ('text.latex.preview : False')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.5.3/matplotlibrc.template
or from the matplotlib source distribution

Bad key mathtext.fallback_to_cm in file C:\Users\spa84\anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle, line 155 ('mathtext.fallback_to_cm : True  # When True, use symbols from the Computer Modern')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.5.3/matplotlibrc.template
or from the matplotlib source distribution

Bad key savefig.jpeg_quality in file C:\Users\spa84\anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle, line 418 ('savefig.jpeg_quality: 95       # when a jpeg is saved, the default quality parameter.')
You proba

In [2]:
warnings.simplefilter('ignore')

In [3]:
font_path = "C:\\Users\\spa84\\Downloads\\text_mining\\NanumGothic.ttf"  # 나눔 폰트의 경로를 지정
font_name = fm.FontProperties(fname=font_path).get_name()
plt.rc("font", family=font_name)

In [4]:
df = pd.read_csv('./data/Before_Encoding_5000.csv', encoding='euc-kr')

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 19 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Building_Age                 5000 non-null   int64  
 1   JS_Price                     5000 non-null   int64  
 2   JS_BA                        5000 non-null   float64
 3   Population                   5000 non-null   int64  
 4   UR                           5000 non-null   float64
 5   LC_index                     5000 non-null   float64
 6   CA_index                     5000 non-null   float64
 7   TC_index                     5000 non-null   float64
 8   SDT_index                    5000 non-null   float64
 9   HSP_index                    5000 non-null   float64
 10  Sell_Price                   5000 non-null   float64
 11  Crime_Rates                  5000 non-null   float64
 12  IR                           5000 non-null   float64
 13  Region_Name       

## Categorical:

- Nominal(variables that have two or more categories, but which do not have an intrinsic order.)

    - Region_Name : 자치구 명
    - Building_Use : 건물 용도
    
- Ordinal(variables that have two or more categories just like nominal variables. Only the categories can also be ordered or ranked.)

    
## Numeric:

- Discrete
    - YearMonth : 년월
    - Building_Age : 건물연식
    - JS_Price : 전세가
   
- Continous
    - Sell_Price : 매매 가격
    - JS_BA = JS_Building Area : 임대 면적
    - lR = Interest Rate : 금리
    - UR = Unemployment Rate : 실업률
    - LC_index = Leading Composite index : 선행종합 지수
    - CA_index = Comprehensive Accompany index : 동행종합 지수
    - TC_index = Trailing Composite index : 후행종합 지수
    - SDT_index = Supply and Demand Trend index = 전세수급동향 지수
    - HSP_index = 
    - Population : 인구수
    - Crime_Rates : 범죄율
    - Shortest_Distance_to_Subway : 가장 가까운 지하철역과의 거리
    - Shortest_Distance_to_School : 가장 가까운 초중고등학교와의 거리
    - Shortest_Distance_to_Univ : 가장 가까운 대학교와의 거리
    - Shortest_Distance_to_Park : 가장 가까운 공원과의 거리
    

## 데이터전처리

### 인코딩

In [6]:
def target_encoding(train_df, test_df, target_column, categorical_columns):
    train_encoded = train_df.copy()
    test_encoded = test_df.copy()
    
    for column in categorical_columns:
        if train_df[column].dtype == object:
            encoding_map = train_df.groupby(column)[target_column].mean().to_dict()
            train_encoded[column] = train_encoded[column].map(encoding_map)
            test_encoded[column] = test_encoded[column].map(encoding_map)
    
    return train_encoded, test_encoded

# 훈련 데이터와 테스트 데이터를 나누기 전에 호출
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['JS_Price_Category'])
train_encoded, test_encoded = target_encoding(train_df, test_df, 'JS_Price', categorical_columns)

KeyError: 'JS_Price_Category'

In [ ]:
df_encoded

### 종속변수 변환

In [ ]:
# JS_Price를 4개의 범주로 나누고 기존 변수 삭제
df['JS_Price_Category'] = pd.cut(df['JS_Price'], bins=5, labels=False)
df.drop('JS_Price', axis=1, inplace=True)

### 오버샘플링
- 클래스 별 샘플 수 차이로 인해 오버샘플링 진행

In [ ]:
df_encoded['JS_Price_Category'].value_counts()

In [ ]:
from imblearn.over_sampling import RandomOverSampler

# 훈련 데이터와 테스트 데이터를 나눈 후에 호출
X_train, y_train = train_encoded.drop('JS_Price_Category', axis=1), train_encoded['JS_Price_Category']
X_test, y_test = test_encoded.drop('JS_Price_Category', axis=1), test_encoded['JS_Price_Category']

# RandomOverSampler를 초기화합니다.
oversampler = RandomOverSampler(sampling_strategy='auto', random_state=42)

# 오버샘플링을 적용합니다. (훈련 데이터에만 적용)
X_train_resampled, y_train_resampled = oversampler.fit_resample(X_train, y_train)

# 오버샘플링된 데이터를 새로운 데이터프레임으로 만듭니다.
train_encoded_resampled = pd.concat([X_train_resampled, y_train_resampled], axis=1)

## Logistic Regression Analysis

In [ ]:
selected_features = ['CA_index',
'Shortest_Distance_to_School',
'HSP_index',
'TC_index',
'IR',
'SDT_index',
'UR']

In [ ]:
# 독립 변수 선택
X = df_encoded[selected_features]

# 종속 변수 선택 (JS_Price_Category, 다중 분류)
Y = df_encoded['JS_Price_Category']

# 층화 추출을 사용하여 데이터 분할
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42, stratify=Y)

# 표준화 (선택적)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# 다항 로지스틱 회귀 모델 생성
model = LogisticRegression(multi_class='multinomial', solver='lbfgs')
model.fit(X_train, Y_train)

# 모델 평가 (정확도 계산)
accuracy = model.score(X_test, Y_test)
print(f"모델 정확도 : {accuracy}", '\n')

# 각 클래스에 속할 확률 예측
probabilities = model.predict_proba(X_test)

# 예측 결과 클래스 (가장 높은 확률을 갖는 클래스 선택)
predicted_classes = model.predict(X_test)

# 예측된 클래스 및 확률 출력
print("---예측된 클래스---", '\n', predicted_classes, '\n')
print("---클래스별 확률---", '\n', probabilities)

### 로지스틱 회귀 분석의 통계량
- 양수 회귀 계수는 종속 변수를 증가시키는 데 긍정적인 영향을 미치고, 음수 회귀 계수는 종속 변수를 감소시키는 데 부정적인 영향을 미칩니다.
- 오즈비가 1보다 크면 해당 독립 변수가 종속 변수의 확률에 긍정적인 영향을 미칩니다. 오즈비가 1보다 작으면 부정적인 영향을 미칩니다.

In [ ]:
# 회귀 모델을 이미 만들었다고 가정합니다.
# model은 이미 다중 클래스 로지스틱 회귀 모델로 훈련되어 있다고 가정합니다.

# 독립 변수의 열 이름
independent_variable_names = selected_features 

# 종속 변수 열의 이름을 식별하고 고유한 클래스 값을 확인합니다.
dependent_variable_column = "JS_Price_Category"
dependent_variable_classes = df_encoded[dependent_variable_column].unique()

# 회귀 계수 출력
coefficients = model.coef_

# 각 독립 변수와 종속 변수 클래스에 대한 회귀 계수와 오즈비를 출력합니다.
for i, dep_class in enumerate(dependent_variable_classes):
    print(f"종속 변수 클래스: {dep_class}")
    for j, indep_var in enumerate(independent_variable_names):
        coef = coefficients[i][j]
        odds_ratio = np.exp(coef)
        print(f"{indep_var} - 회귀 계수: {coef}, 오즈비: {odds_ratio}")

## 다중공선성 체크

In [ ]:
# 독립 변수 데이터프레임을 준비합니다. 여기서는 X_train을 사용합니다.
# X_train은 모델 훈련에 사용된 독립 변수 데이터입니다.
independent_variables = pd.DataFrame(X_train, columns=selected_features)

# VIF 계산을 위한 함수를 정의합니다.
def calculate_vif(data):
    vif_data = pd.DataFrame()
    vif_data["Variable"] = data.columns
    vif_data["VIF"] = [variance_inflation_factor(data.values, i) for i in range(data.shape[1])]
    return vif_data

# VIF를 계산합니다.
vif_result = calculate_vif(independent_variables)

# VIF 결과를 출력합니다.
print(vif_result)

- VIF값 5를 기준으로 높으면 변수에서 제거할 요인으로 본다면 CA_index, TC_index, HSP_index를 제외한다.

## 과적합 체크

In [ ]:
import matplotlib.pyplot as plt
from sklearn.model_selection import learning_curve

# 학습 곡선 생성
train_sizes, train_scores, test_scores = learning_curve(model, X_train, Y_train, cv=5)

# 평균 정확도 계산
train_scores_mean = train_scores.mean(axis=1)
test_scores_mean = test_scores.mean(axis=1)

# 학습 곡선 그리기
plt.figure(figsize=(10, 6))
plt.title('Learning Curve')
plt.xlabel('Training Examples')
plt.ylabel('Accuracy')
plt.grid()

plt.plot(train_sizes, train_scores_mean, label='Training Accuracy')
plt.plot(train_sizes, test_scores_mean, label='Validation Accuracy')
plt.legend(loc='best')

plt.show()

## 변수선택법
- p-value로 최종선택

In [ ]:
# 로지스틱 회귀 모델 생성 (다항 로지스틱 회귀 모델)
model = sm.MNLogit(Y, sm.add_constant(X))  # 상수항 추가

# 모델 피팅
result = model.fit()

# 독립 변수들의 p-value 추출
p_values = result.pvalues
p_values

- a=0.05일 때, p-value값을 고려하면 IR, UR, HSP_index, CA_index을 제외한다.

## 최종결론
- Features_Ranking의 17가지 변수 중 상위 10가지를 제외한 7가지 변수에 대해 VIF와 Logistic Regression Analysis를 진행한다.
    - VIF와 Logistit Regression의 p-value값의 공통 제거요인인 HSP_index, CA_index
    - 전체 p-value에서 가장 높은 값을 가지고 Features_Ranking에서도 최하위에 속하는 UR
    - 위의 세가지 요인(HSP_index, CA_index, UR)을 변수에서 제거하여 머신러닝을 진행한다.